# Transformer with Huggingface
- 이전까지는 모델에 신경썼지만, 이제는 학습된 가중치를 어떻게 사용하느냐
- BERT: transformer 인코더 > MLM & NSP: pre=train 후 fine-tuning 재학습
- 모델을 어떻게 학습시키느냐와 사전학습 weight(더 중요)
- KoBERT: 한국어 자료로 사전학습
- 또한 상황에 따라 모델 크기 다르게 사용 GPT small, large...(디코더레이어 수와 hidden dim size가 다름), 커질수록 더많은 GPU자원 필요
- 모델/가중치마다 tokenizer 다르게 사용